In [7]:
import boto3
from datetime import date, timedelta

In [3]:
client=boto3.client('dynamodb', region_name='us-east-1')

In [11]:
for item in items:
    print(f'putting {item}...')
    response=client.put_item(
        TableName='testes',
        Item=item
    )

    print('done!')


putting {'id': {'N': '3724'}, 'date': {'N': '20240424'}}...
done!
putting {'id': {'N': '6124'}, 'date': {'N': '20240325'}, 'another_id': {'N': '3564'}, 'thats_an_attr': {'S': 'PUyydfFPtUeAGtRu'}}...
done!
putting {'id': {'N': '4697'}, 'date': {'N': '20240413'}, 'another_id': {'N': '8024'}, 'thats_an_attr': {'S': 'jSyfTuIvfntnjpvN'}}...
done!
putting {'id': {'N': '4795'}, 'date': {'N': '20240106'}, 'another_id': {'N': '5302'}}...
done!
putting {'id': {'N': '2478'}, 'date': {'N': '20240218'}, 'thats_an_attr': {'S': 'LMViSNjkujVdxRsO'}}...
done!
putting {'id': {'N': '7494'}, 'date': {'N': '20240409'}, 'thats_an_attr': {'S': 'qoXnqjvJRWnpRHgp'}}...
done!
putting {'id': {'N': '8990'}, 'date': {'N': '20240319'}}...
done!
putting {'id': {'N': '7967'}, 'date': {'N': '20240109'}}...
done!
putting {'id': {'N': '6431'}, 'date': {'N': '20240214'}, 'another_id': {'N': '5741'}}...
done!
putting {'id': {'N': '5338'}, 'date': {'N': '20240421'}, 'thats_an_attr': {'S': 'XqyISGOEXctIdteQ'}}...
done!
putt

In [58]:
#yesterday=(date.today()-timedelta(days=1)).strftime("%Y%m%d")
start_date='20240101'
end_date='20240229'

print(f'getting data for {start_date} to {end_date}... ')

scan_kwargs = {
    "TableName": "testes", #
    #"ProjectionExpression": "#another_id, #date",
    #"Select": "SPECIFIC_ATTRIBUTES",
    "FilterExpression": "attribute_exists(#another_id) AND (#date >= :start_date AND #date <= :end_date)",
    "ExpressionAttributeNames": {"#date": "date", "#another_id": "another_id"},
    "ExpressionAttributeValues": {":start_date": {"N": f"{start_date}"}, ":end_date": {"N": f"{end_date}"}}
}

done = False
start_key = None
count_item=0
from_scan=[]
while not done:
    if start_key:
        scan_kwargs["ExclusiveStartKey"] = start_key
    response=client.scan(**scan_kwargs)

    print(f'scaning {count_item} to {response.get("ScannedCount")}... ')
    from_scan.extend(response.get("Items", []))

    count_item += response.get("Count")
    start_key = response.get("LastEvaluatedKey", None)
    done = start_key is None

print('done!')

getting data for 20240101 to 20240229... 
scaning 0 to 100... 
done!


In [60]:
response

{'Items': [{'id': {'N': '5967'},
   'another_id': {'N': '6569'},
   'date': {'N': '20240131'}},
  {'id': {'N': '2801'},
   'another_id': {'N': '8521'},
   'date': {'N': '20240205'}},
  {'id': {'N': '8915'},
   'another_id': {'N': '4803'},
   'date': {'N': '20240229'}},
  {'thats_an_attr': {'S': 'LouEFbCTyVRGweds'},
   'another_id': {'N': '3092'},
   'date': {'N': '20240209'},
   'id': {'N': '9707'}},
  {'thats_an_attr': {'S': 'NRDSoDeetJwpBFvp'},
   'another_id': {'N': '7753'},
   'date': {'N': '20240113'},
   'id': {'N': '7558'}},
  {'id': {'N': '3626'},
   'another_id': {'N': '9583'},
   'date': {'N': '20240224'}},
  {'thats_an_attr': {'S': 'LPEtwpfidohPqzcu'},
   'another_id': {'N': '6083'},
   'date': {'N': '20240201'},
   'id': {'N': '9548'}},
  {'id': {'N': '3244'},
   'another_id': {'N': '4555'},
   'date': {'N': '20240228'}},
  {'id': {'N': '8862'},
   'another_id': {'N': '6481'},
   'date': {'N': '20240218'}},
  {'id': {'N': '1555'},
   'another_id': {'N': '1108'},
   'date': 

In [59]:
from_scan

[{'id': {'N': '5967'}, 'another_id': {'N': '6569'}, 'date': {'N': '20240131'}},
 {'id': {'N': '2801'}, 'another_id': {'N': '8521'}, 'date': {'N': '20240205'}},
 {'id': {'N': '8915'}, 'another_id': {'N': '4803'}, 'date': {'N': '20240229'}},
 {'thats_an_attr': {'S': 'LouEFbCTyVRGweds'},
  'another_id': {'N': '3092'},
  'date': {'N': '20240209'},
  'id': {'N': '9707'}},
 {'thats_an_attr': {'S': 'NRDSoDeetJwpBFvp'},
  'another_id': {'N': '7753'},
  'date': {'N': '20240113'},
  'id': {'N': '7558'}},
 {'id': {'N': '3626'}, 'another_id': {'N': '9583'}, 'date': {'N': '20240224'}},
 {'thats_an_attr': {'S': 'LPEtwpfidohPqzcu'},
  'another_id': {'N': '6083'},
  'date': {'N': '20240201'},
  'id': {'N': '9548'}},
 {'id': {'N': '3244'}, 'another_id': {'N': '4555'}, 'date': {'N': '20240228'}},
 {'id': {'N': '8862'}, 'another_id': {'N': '6481'}, 'date': {'N': '20240218'}},
 {'id': {'N': '1555'}, 'another_id': {'N': '1108'}, 'date': {'N': '20240112'}},
 {'id': {'N': '7911'}, 'another_id': {'N': '8416'}

In [35]:
query_kwargs={
    "TableName": "testes",
    "KeyConditionExpression": "#id = :id",
    "ExpressionAttributeNames": {"#id": "id"},
    "ExpressionAttributeValues": {":id": {"N": f"7860"}}}
#response_of_query=client.query(**query_kwargs)

done = False
start_key = None
count_item=0
from_query=[]
while not done:
    if start_key:
        scan_kwargs["ExclusiveStartKey"] = start_key
    response_of_query=client.query(**query_kwargs)

    print(f'querying {count_item} to {response.get("ScannedCount")}... ')
    from_query.extend(response.get("Items", []))

    count_item += response.get("Count")
    start_key = response.get("LastEvaluatedKey", None)
    done = start_key is None

print('done!')

querying 0 to 100... 
done!


In [55]:
response_of_query

{'Items': [{'id': {'N': '7860'},
   'another_id': {'N': '9678'},
   'date': {'N': '20240223'}}],
 'Count': 1,
 'ScannedCount': 1,
 'ResponseMetadata': {'RequestId': 'B2VAT3OA4A5P8UT9DUO7O5HDKRVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 16 Oct 2024 16:37:11 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '108',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'B2VAT3OA4A5P8UT9DUO7O5HDKRVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '60540292'},
  'RetryAttempts': 0}}

In [36]:
from_query

[{'id': {'N': '5967'}, 'another_id': {'N': '6569'}, 'date': {'N': '20240131'}},
 {'id': {'N': '2801'}, 'another_id': {'N': '8521'}, 'date': {'N': '20240205'}},
 {'id': {'N': '8528'}, 'date': {'N': '20240112'}},
 {'id': {'N': '7314'}, 'date': {'N': '20240309'}},
 {'id': {'N': '3925'}, 'date': {'N': '20240105'}},
 {'thats_an_attr': {'S': 'salJhLbETVvxWAtD'},
  'another_id': {'N': '8676'},
  'date': {'N': '20240426'},
  'id': {'N': '7722'}},
 {'id': {'N': '1687'}, 'another_id': {'N': '7712'}, 'date': {'N': '20240402'}},
 {'id': {'N': '7970'},
  'thats_an_attr': {'S': 'nPFmAZHfOuEgEjxP'},
  'date': {'N': '20240424'}},
 {'id': {'N': '8915'}, 'another_id': {'N': '4803'}, 'date': {'N': '20240229'}},
 {'id': {'N': '1171'}, 'another_id': {'N': '1445'}, 'date': {'N': '20240305'}},
 {'id': {'N': '5264'},
  'thats_an_attr': {'S': 'uijoXbYwlvbRbBzu'},
  'date': {'N': '20240213'}},
 {'id': {'N': '1984'}, 'date': {'N': '20240222'}},
 {'thats_an_attr': {'S': 'LouEFbCTyVRGweds'},
  'another_id': {'N': 